In [1]:

import sys
import os
import yaml

sys.path.append(os.getenv("CODE_PATH"))
sys.path.append(os.getenv("FIN_DATABASE_PATH"))


import plotly.graph_objects as go
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import logging
import matplotlib.pyplot as plt
from dotenv import load_dotenv
import ta
from log_config import setup_logging
from Data.connect import engine, DailyStockData, HourlyStockData, OneMinuteStockData, FiveMinuteStockData,FifteenMinuteStockData, StockSplits, StockNews, CompanyFinancials
from Pre_Processing.pre_processing import PreProcessing
from Feature_Engineering.feature_engineering import TechnicalIndicators
from pipeline import Pipeline

from sklearn.metrics import roc_auc_score, roc_curve, auc

from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, roc_curve, auc, log_loss, confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import TimeSeriesSplit

In [ ]:
# wiki = 'http://en.wikipedia.org/wiki'
# djia_ticker_list = wiki + '/Dow_Jones_Industrial_Average'
# sp500_tickers_list = wiki + '/List_of_S%26P_500_companies'
# tickersSP500 = pd.read_html(sp500_tickers_list)[0].Symbol.to_list()
# djia_tickers = pd.read_html(djia_ticker_list)[1].Symbol.to_list()

In [ ]:
# tickers = ['AAPL', 'MSFT', 'DIS', 'V', 'JPM']

In [2]:
tickers = ['AAPL', 'MSFT', 'SPY']

In [3]:
pipe = Pipeline(tickers)

In [4]:
data = pipe.pipeline()

In [5]:
for df in data.values():
    df['target'] = np.where(df['log_ret'] > 0, 1, 0)
    df['target'] = df['target'].shift(-1)
    df.dropna(inplace=True)

In [ ]:
# #Combining the data into a single dataframe

# combined_df = pd.concat(
#     [df.assign(ticker=ticker).set_index('ticker', append=True) for ticker, df in data.items()]
# )

# combined_df = combined_df.reorder_levels(['date', 'ticker'])


# combined_df.index = combined_df.index.set_levels(pd.to_datetime(combined_df.index.levels[0]), level=0)

# combined_df.sort_index(inplace=True)

# combined_df.columns = combined_df.columns.str.lower()
# combined_df.dropna(subset='target', inplace=True)
# combined_df

In [ ]:
# combined_df['target'] = np.where(combined_df['log_ret'] > 0,1,
#                                 0)
# combined_df['target'] = combined_df['target'].shift(-1)

## Models
Applying Decision Tree models

In [7]:
data['AAPL']

,id,date,timestamp,ticker,open,high,low,close,volume,vwap,...,sma_100,ema_100,sma_200,ema_200,sma_300,ema_300,day,month,day_of_week,target
date,,,,,,,,,,,,,,,,,,,,,
2004-11-16,211969,2004-11-16,1100581200000,AAPL,0.9850,0.9857,0.9729,0.9810,2.952198e+08,0.9783,...,0.680400,0.722843,0.580693,0.622927,0.518192,0.569883,16,11,1,0.0
2004-11-17,211970,2004-11-17,1100667600000,AAPL,0.9856,0.9902,0.9682,0.9804,3.987650e+08,0.9825,...,0.684402,0.727943,0.583608,0.626484,0.520140,0.572611,17,11,2,1.0
2004-11-18,211971,2004-11-18,1100754000000,AAPL,0.9697,0.9902,0.9695,0.9891,4.551612e+08,0.9865,...,0.688489,0.733115,0.586607,0.630092,0.522094,0.575378,18,11,3,0.0
2004-11-19,211972,2004-11-19,1100840400000,AAPL,0.9909,1.0163,0.9732,0.9852,7.642511e+08,0.9911,...,0.692530,0.738106,0.589531,0.633626,0.524003,0.578101,19,11,4,1.0
2004-11-22,211973,2004-11-22,1101099600000,AAPL,1.0361,1.1429,1.0340,1.0956,2.563821e+09,1.0904,...,0.697718,0.745185,0.592982,0.638222,0.526333,0.581540,22,11,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-02,216929,2024-08-02,1722571200000,AAPL,219.1500,225.6000,217.7100,219.8600,1.026353e+08,221.9519,...,193.754300,202.203530,189.521550,192.500660,187.166483,186.473798,2,8,4,0.0
2024-08-05,216930,2024-08-05,1722830400000,AAPL,199.0900,213.5000,196.0000,209.2700,1.193825e+08,207.8010,...,194.114700,202.343460,189.682150,192.667519,187.291250,186.625268,5,8,0,0.0
2024-08-06,216931,2024-08-06,1722916800000,AAPL,205.3000,209.9900,201.0700,207.2300,6.954539e+07,206.7390,...,194.475700,202.440223,189.839100,192.812419,187.405383,186.762176,6,8,1,1.0


In [ ]:
features = [ 'open', 'high', 'low', 'close',
       'volume', 'vwap', 'RSI_14', 'RSI_2', 'MACD', 'log_ret',
       'ROC', 'Stoch', 'ADX', 'ADX_pos', 'ADX_neg', 'rolling_H-L_25',
       'lower_band', 'ATR', 'IBS', 'OBV', 'ema_5', 'ema_10',
        'ema_30', 'ema_50', 'ema_100',
       'ema_200', 'ema_300']
target = 'target'

## Cross Validation

In [ ]:
#Testing different classifiers

models = [
    ("XGBoost", XGBClassifier()),
    ("CatBoost", CatBoostClassifier(verbose=False)),
    ("RandomForest", RandomForestClassifier())
]

In [ ]:
tcsv= TimeSeriesSplit(n_splits=5)

In [ ]:
roc_auc_scores = {model_name: {} for model_name, _ in models}
log_loss_scores = {model_name: {} for model_name, _ in models}

In [ ]:
results = {model_name: {} for model_name, _ in models}

for ticker, df in data.items():
    for model_name, _ in models:
        results[model_name][ticker] = {
            'roc_auc': [],
            'log_loss': []
        }
    
    fold_number = 1
    for train_index, test_index in tcsv.split(df):
        X_train, X_test = df[features].iloc[train_index], df[features].iloc[test_index]
        y_train, y_test = df['target'].iloc[train_index], df['target'].iloc[test_index]
        
        for model_name, model in models:
            model.fit(X_train, y_train)
            y_proba = model.predict_proba(X_test)[:, 1]
            
            roc_auc = roc_auc_score(y_test, y_proba)
            log_loss_value = log_loss(y_test, model.predict_proba(X_test))
            
            # Store the results for each fold
            results[model_name][ticker]['roc_auc'].append((fold_number, roc_auc))
            results[model_name][ticker]['log_loss'].append((fold_number, log_loss_value))
        
        fold_number += 1


In [ ]:
for model_name, tickers in results.items():
    print(f"Results for Model: {model_name}")
    for ticker, metrics in tickers.items():
        print(f"  Ticker: {ticker}")
        for fold, roc_auc in metrics['roc_auc']:
            log_loss_value = [ll for f, ll in metrics['log_loss'] if f == fold][0]
            print(f"    Fold {fold}: ROC AUC = {roc_auc:.4f}, Log Loss = {log_loss_value:.4f}")
        print()
